In [1]:
import pickle
import random
from collections import namedtuple

import gym
import numpy as np
import ptan
import torch
import torch.optim as optim
from ptan.agent import float32_preprocessor

from util import PGN, RewardNet

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from matplotlib import pyplot as plt

In [2]:
GAMMA = 0.99
LEARNING_RATE = 0.01
EPISODES_TO_TRAIN = 4
DEMO_BATCH = 50
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

EpisodeStep = namedtuple('EpisodeStep', field_names=['state', 'action', 'reward', 'next_state'])
Trajectory = namedtuple('Trajectory', field_names=['prob', 'episode_steps'])

In [3]:
def calc_qvals(rewards):
    res = []
    sum_r = 0.0
    for r in reversed(rewards):
        sum_r *= GAMMA
        sum_r += r
        res.append(sum_r)
    return list(reversed(res))


def process_demonstrations(demo_samples):
    traj_states, traj_actions, traj_qvals, traj_prob = [], [], [], []
    for traj in demo_samples:
        states, actions, rewards, qvals = [], [], [], []
        traj_prob.append(traj.prob)
        for step in traj.episode_steps:
            states.append(step.state)
            actions.append(step.action)
            rewards.append(step.reward)
        qvals.extend(calc_qvals(rewards))

        traj_states.append(states)
        traj_actions.append(actions)
        traj_qvals.append(qvals)
    traj_states = np.array(traj_states, dtype=np.object)
    traj_actions = np.array(traj_actions, dtype=np.object)
    traj_qvals = np.array(traj_qvals, dtype=np.object)
    traj_prob = np.array(traj_prob, dtype=np.float)
    return {'states': traj_states, 'actions': traj_actions, 'qvals': traj_qvals, 'traj_probs': traj_prob}

In [4]:
env = gym.make('CartPole-v1')
agent_net = PGN(env.observation_space.shape[0], env.action_space.n)
reward_net = RewardNet(env.observation_space.shape[0] + 1)
agent = ptan.agent.PolicyAgent(agent_net, preprocessor=float32_preprocessor, apply_softmax=True)
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA)
optimizer_agent = optim.Adam(agent_net.parameters(), lr=LEARNING_RATE)
optimizer_reward = optim.Adam(reward_net.parameters(), lr=1e-2, weight_decay=1e-4)

In [5]:
with open('demonstrations.list.pkl', 'rb') as f:
    demonstrations = pickle.load(f)
assert (len(demonstrations) > DEMO_BATCH)
print(f'Number of demonstrations: {len(demonstrations)}')
demonstrations = process_demonstrations(demonstrations)

Number of demonstrations: 100


In [6]:
total_rewards = []
step_idx = 0
done_episodes = 0

batch_episodes = 0
batch_states, batch_actions, batch_qvals = [], [], []
cur_rewards = []
loss_rwd = 0.

for step_idx, exp in enumerate(exp_source):
    batch_states.append(exp.state)
    batch_actions.append(int(exp.action))
    x = torch.cat([float32_preprocessor(exp.state), float32_preprocessor([int(exp.action)])]).view(1, -1)
    reward = reward_net(x)
    cur_rewards.append(reward.item())

    if exp.last_state is None:
        batch_qvals.extend(calc_qvals(cur_rewards))
        cur_rewards.clear()
        batch_episodes += 1

    new_rewards = exp_source.pop_total_rewards()
    if new_rewards:
        done_episodes += 1
        reward = new_rewards[0]
        total_rewards.append(reward)
        mean_rewards = float(np.mean(total_rewards[-100:]))
        writer.add_scalar('reward', reward, done_episodes)
        writer.add_scalar('mean_reward', mean_rewards, done_episodes)
        writer.add_scalar('loss_rwd', loss_rwd, done_episodes) 
        print(f'{step_idx}: reward: {reward:6.2f}, mean_100: {mean_rewards:6.2f}, '
              f'episodes: {done_episodes}, reward function loss: {loss_rwd:6.4f}')
        
        if done_episodes%100 == 0 or mean_rewards>=500:
            N_samp = 20
            S1 = np.linspace(-5, 5, N_samp)
            S2 = np.linspace(-3.1457, 3.1457, N_samp)
            S3 = 0*S1
            S4 = 0*S1
            
            S5 = np.ones(N_samp)
            
            for action in [0,1]:
                Reward = np.zeros((N_samp,N_samp))
                for i in range(N_samp):
                    for j in range(N_samp):
                        state = [S1[i], S2[j], 0, 0]
#                         action = 1
                        x = torch.cat([float32_preprocessor(state), float32_preprocessor([int(action)])]).view(1, -1)
                        r = reward_net(x)
                        Reward[i,j] = r

                X, Y = np.meshgrid(S1, S2)
                Z = Reward
                fig = plt.figure()
                ax = plt.axes(projection='3d')
                ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                                cmap='viridis', edgecolor='none')
                ax.set_title('surface');
                ax.set_xlabel('x')
                ax.set_ylabel('y')
                ax.set_zlabel('z');
                ax.view_init(azim=90, elev=90)
                if action ==0:
                    writer.add_figure('x1 vs x2 with a = 0', fig, global_step=done_episodes/100)
                if action ==1:
                    writer.add_figure('x1 vs x2 with a = 1', fig, global_step=done_episodes/100)    
            
        if mean_rewards >= 600:
            print(f'Solved in {step_idx} steps and {done_episodes} episodes!')
            torch.save(agent_net.state_dict(), 'cartpole_learner.mod')
            torch.save(reward_net.state_dict(), 'cartpole-v1_reward_func.mod')
            break
        

    if batch_episodes < EPISODES_TO_TRAIN:
        continue

    states_v = torch.FloatTensor(batch_states)
    batch_actions_t = torch.LongTensor(batch_actions)
    batch_qvals_v = torch.FloatTensor(batch_qvals)

    # reward function learning
    demo_states = demonstrations['states']
    demo_actions = demonstrations['actions']
    demo_probs = demonstrations['traj_probs']
    for rf_i in range(10):
        # Check is preprocessing is required 
        selected = np.random.choice(len(demonstrations), DEMO_BATCH)
#         print(selected)
        demo_states = demo_states[selected]
        demo_actions = demo_actions[selected]
#         demo_probs = demo_probs[selected]
        demo_batch_states, demo_batch_actions = [], []
        for idx in range(len(demo_states)):
            demo_batch_states.extend(demo_states[idx])
            demo_batch_actions.extend(demo_actions[idx])
        demo_batch_states = torch.FloatTensor(demo_batch_states)
        demo_batch_actions = torch.FloatTensor(demo_batch_actions)
        D_demo = torch.cat([demo_batch_states, demo_batch_actions.view(-1, 1)], dim=-1)
        D_samp = torch.cat([states_v, batch_actions_t.float().view(-1, 1)], dim=-1)
        D_samp = torch.cat([D_demo, D_samp])
        # dummy importance weights - fix later
        z = torch.ones((D_samp.shape[0], 1))

        # objective
        D_demo_out = reward_net(D_demo)
        D_samp_out = reward_net(D_samp)
        D_samp_out = z * torch.exp(D_samp_out)
        loss_rwd = torch.mean(D_demo_out) - torch.log(torch.mean(D_samp_out))
        loss_rwd = -loss_rwd  # for maximization

        # update parameters
        optimizer_reward.zero_grad()
        loss_rwd.backward()
        optimizer_reward.step()

    # agent
    optimizer_agent.zero_grad()
    logits_v = agent_net(states_v)
    log_prob_v = torch.log_softmax(logits_v, dim=1)
    # REINFORCE
    log_prob_actions_v = batch_qvals_v * log_prob_v[range(len(batch_states)), batch_actions_t]
    loss_v = -log_prob_actions_v.mean()
    
    writer.add_scalar('loss_agent_net', loss_v, done_episodes) 
    
    loss_v.backward()
    optimizer_agent.step()

    batch_episodes = 0
    batch_states.clear()
    batch_actions.clear()
    batch_qvals.clear()
env.close()
writer.close()

10: reward:  10.00, mean_100:  10.00, episodes: 1, reward function loss: 0.0000
23: reward:  13.00, mean_100:  11.50, episodes: 2, reward function loss: 0.0000
38: reward:  15.00, mean_100:  12.67, episodes: 3, reward function loss: 0.0000
72: reward:  34.00, mean_100:  18.00, episodes: 4, reward function loss: 0.0001
84: reward:  12.00, mean_100:  16.80, episodes: 5, reward function loss: 0.0001
101: reward:  17.00, mean_100:  16.83, episodes: 6, reward function loss: 0.0001
112: reward:  11.00, mean_100:  16.00, episodes: 7, reward function loss: 0.0001
142: reward:  30.00, mean_100:  17.75, episodes: 8, reward function loss: -0.0000
161: reward:  19.00, mean_100:  17.89, episodes: 9, reward function loss: -0.0000
177: reward:  16.00, mean_100:  17.70, episodes: 10, reward function loss: -0.0000
186: reward:   9.00, mean_100:  16.91, episodes: 11, reward function loss: -0.0000
205: reward:  19.00, mean_100:  17.08, episodes: 12, reward function loss: -0.0000
216: reward:  11.00, mean

2945: reward:  36.00, mean_100:  29.45, episodes: 100, reward function loss: -0.0009
2987: reward:  42.00, mean_100:  29.77, episodes: 101, reward function loss: -0.0009
3002: reward:  15.00, mean_100:  29.79, episodes: 102, reward function loss: -0.0009
3082: reward:  80.00, mean_100:  30.44, episodes: 103, reward function loss: -0.0009
3134: reward:  52.00, mean_100:  30.62, episodes: 104, reward function loss: -0.0000
3174: reward:  40.00, mean_100:  30.90, episodes: 105, reward function loss: -0.0000
3288: reward: 114.00, mean_100:  31.87, episodes: 106, reward function loss: -0.0000
3335: reward:  47.00, mean_100:  32.23, episodes: 107, reward function loss: -0.0000
3450: reward: 115.00, mean_100:  33.08, episodes: 108, reward function loss: -0.0004
3516: reward:  66.00, mean_100:  33.55, episodes: 109, reward function loss: -0.0004
3565: reward:  49.00, mean_100:  33.88, episodes: 110, reward function loss: -0.0004
3635: reward:  70.00, mean_100:  34.49, episodes: 111, reward fun

21887: reward: 500.00, mean_100: 190.97, episodes: 197, reward function loss: -0.0005
22163: reward: 276.00, mean_100: 192.96, episodes: 198, reward function loss: -0.0005
22620: reward: 457.00, mean_100: 197.11, episodes: 199, reward function loss: -0.0005
23010: reward: 390.00, mean_100: 200.65, episodes: 200, reward function loss: -0.0001
23510: reward: 500.00, mean_100: 205.23, episodes: 201, reward function loss: -0.0001
24010: reward: 500.00, mean_100: 210.08, episodes: 202, reward function loss: -0.0001
24324: reward: 314.00, mean_100: 212.42, episodes: 203, reward function loss: -0.0001
24824: reward: 500.00, mean_100: 216.90, episodes: 204, reward function loss: -0.0000
24903: reward:  79.00, mean_100: 217.29, episodes: 205, reward function loss: -0.0000
25403: reward: 500.00, mean_100: 221.15, episodes: 206, reward function loss: -0.0000
25858: reward: 455.00, mean_100: 225.23, episodes: 207, reward function loss: -0.0000
26358: reward: 500.00, mean_100: 229.08, episodes: 208

41432: reward: 131.00, mean_100: 200.45, episodes: 296, reward function loss: -0.0004
41543: reward: 111.00, mean_100: 196.56, episodes: 297, reward function loss: -0.0004
41645: reward: 102.00, mean_100: 194.82, episodes: 298, reward function loss: -0.0004
41767: reward: 122.00, mean_100: 191.47, episodes: 299, reward function loss: -0.0004
41871: reward: 104.00, mean_100: 188.61, episodes: 300, reward function loss: -0.0017
41896: reward:  25.00, mean_100: 183.86, episodes: 301, reward function loss: -0.0017
42007: reward: 111.00, mean_100: 179.97, episodes: 302, reward function loss: -0.0017
42113: reward: 106.00, mean_100: 177.89, episodes: 303, reward function loss: -0.0017
42138: reward:  25.00, mean_100: 173.14, episodes: 304, reward function loss: -0.0017
42166: reward:  28.00, mean_100: 172.63, episodes: 305, reward function loss: -0.0017
42278: reward: 112.00, mean_100: 168.75, episodes: 306, reward function loss: -0.0017
42299: reward:  21.00, mean_100: 164.41, episodes: 307

49974: reward: 139.00, mean_100:  89.51, episodes: 392, reward function loss: -0.0110
50119: reward: 145.00, mean_100:  89.76, episodes: 393, reward function loss: -0.0110
50253: reward: 134.00, mean_100:  89.83, episodes: 394, reward function loss: -0.0110
50400: reward: 147.00, mean_100:  90.99, episodes: 395, reward function loss: -0.0110
50528: reward: 128.00, mean_100:  90.96, episodes: 396, reward function loss: -0.0115
50673: reward: 145.00, mean_100:  91.30, episodes: 397, reward function loss: -0.0115
50808: reward: 135.00, mean_100:  91.63, episodes: 398, reward function loss: -0.0115
50952: reward: 144.00, mean_100:  91.85, episodes: 399, reward function loss: -0.0115
51094: reward: 142.00, mean_100:  92.23, episodes: 400, reward function loss: -0.0104
51237: reward: 143.00, mean_100:  93.41, episodes: 401, reward function loss: -0.0104
51365: reward: 128.00, mean_100:  93.58, episodes: 402, reward function loss: -0.0104
51516: reward: 151.00, mean_100:  94.03, episodes: 403

...

1331032: reward: 500.00, mean_100: 500.00, episodes: 3308, reward function loss: -0.0471
1331532: reward: 500.00, mean_100: 500.00, episodes: 3309, reward function loss: -0.0471
1332032: reward: 500.00, mean_100: 500.00, episodes: 3310, reward function loss: -0.0471
1332532: reward: 500.00, mean_100: 500.00, episodes: 3311, reward function loss: -0.0471
1333032: reward: 500.00, mean_100: 500.00, episodes: 3312, reward function loss: -0.0427
1333532: reward: 500.00, mean_100: 500.00, episodes: 3313, reward function loss: -0.0427
1334032: reward: 500.00, mean_100: 500.00, episodes: 3314, reward function loss: -0.0427
1334532: reward: 500.00, mean_100: 500.00, episodes: 3315, reward function loss: -0.0427
1335032: reward: 500.00, mean_100: 500.00, episodes: 3316, reward function loss: -0.0415
1335532: reward: 500.00, mean_100: 500.00, episodes: 3317, reward function loss: -0.0415
1336032: reward: 500.00, mean_100: 500.00, episodes: 3318, reward function loss: -0.0415
1336532: reward: 500.

1377532: reward: 500.00, mean_100: 500.00, episodes: 3401, reward function loss: -0.0317
1378032: reward: 500.00, mean_100: 500.00, episodes: 3402, reward function loss: -0.0317
1378532: reward: 500.00, mean_100: 500.00, episodes: 3403, reward function loss: -0.0317
1378577: reward:  45.00, mean_100: 495.45, episodes: 3404, reward function loss: -0.0211
1379077: reward: 500.00, mean_100: 495.45, episodes: 3405, reward function loss: -0.0211
1379577: reward: 500.00, mean_100: 495.45, episodes: 3406, reward function loss: -0.0211
1380077: reward: 500.00, mean_100: 495.45, episodes: 3407, reward function loss: -0.0211
1380577: reward: 500.00, mean_100: 495.45, episodes: 3408, reward function loss: -0.0065
1380855: reward: 278.00, mean_100: 493.23, episodes: 3409, reward function loss: -0.0065
1381355: reward: 500.00, mean_100: 493.23, episodes: 3410, reward function loss: -0.0065
1381855: reward: 500.00, mean_100: 493.23, episodes: 3411, reward function loss: -0.0065
1382355: reward: 500.

1389523: reward:  10.00, mean_100: 144.91, episodes: 3496, reward function loss: -0.0000
1389533: reward:  10.00, mean_100: 140.01, episodes: 3497, reward function loss: -0.0000
1389543: reward:  10.00, mean_100: 135.11, episodes: 3498, reward function loss: -0.0000
1389552: reward:   9.00, mean_100: 130.20, episodes: 3499, reward function loss: -0.0000
1389563: reward:  11.00, mean_100: 125.31, episodes: 3500, reward function loss: -0.0000
1389572: reward:   9.00, mean_100: 120.40, episodes: 3501, reward function loss: -0.0000
1389581: reward:   9.00, mean_100: 115.49, episodes: 3502, reward function loss: -0.0000
1389591: reward:  10.00, mean_100: 110.59, episodes: 3503, reward function loss: -0.0000
1389697: reward: 106.00, mean_100: 111.20, episodes: 3504, reward function loss: -0.0000
1389706: reward:   9.00, mean_100: 106.29, episodes: 3505, reward function loss: -0.0000
1389715: reward:   9.00, mean_100: 101.38, episodes: 3506, reward function loss: -0.0000
1389725: reward:  10.

1390531: reward:  10.00, mean_100:  10.46, episodes: 3592, reward function loss: -0.0000
1390540: reward:   9.00, mean_100:  10.45, episodes: 3593, reward function loss: -0.0000
1390549: reward:   9.00, mean_100:  10.45, episodes: 3594, reward function loss: -0.0000
1390557: reward:   8.00, mean_100:  10.44, episodes: 3595, reward function loss: -0.0000
1390567: reward:  10.00, mean_100:  10.44, episodes: 3596, reward function loss: -0.0000
1390576: reward:   9.00, mean_100:  10.43, episodes: 3597, reward function loss: -0.0000
1390586: reward:  10.00, mean_100:  10.43, episodes: 3598, reward function loss: -0.0000
1390596: reward:  10.00, mean_100:  10.44, episodes: 3599, reward function loss: -0.0000
1390605: reward:   9.00, mean_100:  10.42, episodes: 3600, reward function loss: -0.0000
1390614: reward:   9.00, mean_100:  10.42, episodes: 3601, reward function loss: -0.0000
1390624: reward:  10.00, mean_100:  10.43, episodes: 3602, reward function loss: -0.0000
1390633: reward:   9.

KeyboardInterrupt: 

In [ ]:
## Testing 
from util import Agent
agent_net.eval()
agent_ = Agent(agent_net, apply_softmax=True, preprocessor=ptan.agent.float32_preprocessor)

for i in range(10):
    state = env.reset()
    Reward = 0
    done = False
    while not done:
        env.render()
        action = agent_(state)
        new_state, reward, done, _ = env.step(int(action))
        Reward += reward
    print("Trial :", i, " Reward: ", Reward)
env.close()

In [ ]:
selected, len(demonstrations), DEMO_BATCH

In [ ]:
%matplotlib
S1 = np.linspace(-5, 5, 100)
S2 = np.linspace(-3.1457, 3.1457, 100)
S3 = 0*S1
S4 = 0*S1
S5 = np.ones(100)
Reward = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        state = [S1[i], S2[j], 0, 0]
        action = 1
        x = torch.cat([float32_preprocessor(state), float32_preprocessor([int(action)])]).view(1, -1)
        r = reward_net(x)
        Reward[i,j] = r
        
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
ax.set_title('surface');
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');
ax.view_init(azim=90, elev=90)

In [ ]:
demonstrations['states'][0]